In [1]:
import os
import time
import logging
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

# PCA

In [2]:
from sklearn.decomposition import PCA

## miniImagenet + DC_augmented data

In [3]:
# load one run of miniImagenet data
miniImagenet_support_data = pd.read_csv('./data/miniImagenet/support_run_0.csv')
miniImagenet_query_data = pd.read_csv('./data/miniImagenet/query_run_0.csv')
miniImagenet_full = pd.read_csv('./data/miniImagenet/X_aug_run_0.csv')

# prepare train and query data: here train is the support
train_cols = miniImagenet_support_data.columns.difference(['label'])
target_col = 'label'
# x_train_df, y_train_df = miniImagenet_support_data[train_cols], miniImagenet_support_data[target_col]
x_train, y_train = miniImagenet_full[train_cols].values, miniImagenet_full[target_col]
# x_query_df,  y_query_df = miniImagenet_query_data[train_cols], miniImagenet_query_data[target_col]
x_query,  y_query = miniImagenet_query_data[train_cols].values, miniImagenet_query_data[target_col]

In [5]:
# initialize pca
num_samples, num_feats = x_train.shape
NUM_COMPONENTS = 5
pca = PCA(n_components=NUM_COMPONENTS)

# fit and transform training data with pca
x_train_transformed = pca.fit_transform(x_train)

print(f"Percentage of variance explained by each of the {pca.n_components_} components: {pca.explained_variance_ratio_}")

Percentage of variance explained by each of the 5 components: [0.97426581 0.00550326 0.00370029 0.00330253 0.00189194]


In [6]:
x_train_transformed.shape

(3755, 5)

In [11]:
# standardize the new features
x_train_transformed = (x_train_transformed - x_train_transformed.mean())/ x_train_transformed.std()

In [13]:
# train a classifier: logistic regression classifier
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(max_iter=1000).fit(X=x_train_transformed, y=y_train)

In [14]:
## predict test data

# transform new data using already fitted pca (don't re-fit the pca)
x_query_transformed = pca.transform(x_query)

# predict labels using the trained classifier
pred_labels = lr_classifier.predict(x_query_transformed)

# accuracy of the test data
print(f"accuracy of the query data: {np.mean(pred_labels == y_query)}")

accuracy of the query data: 0.64


## Iris dataset

In [ ]:
from sklearn.datasets import load_iris

# load data
iris = load_iris()

# training data: the first 100 points in the iris dataset
x_train, y_train = iris.data[:100], iris.target[:100]
x_test,  y_test = iris.data[100:], iris.target[100:]

# initialize pca
num_samples, num_feats = x_train.shape
num_components = min(num_samples, num_feats)
pca = PCA(n_components=num_components)

# fit and transform training data with pca
X_transformed = pca.fit_transform(x_train)

classifier = DecisionTreeClassifier()
classifier.fit(X_transformed, y_train)
print(f"Percentage of variance explained by each of the {num_components} components: {pca.explained_variance_ratio_}")

## predict test data

# transform new data using already fitted pca (don't re-fit the pca)
x_test_transformed = pca.transform(x_test)

# predict labels using the trained classifier
pred_labels = classifier.predict(x_test_transformed)

# accuracy of the test data
print(f"accuracy of the test data: {sum(pred_labels == y_test)}")

# Lasso

In [ ]:
from sklearn import feature_selection
from sklearn import linear_model

In [ ]:
alpha=0.1
selector = feature_selection.SelectFromModel(estimator=linear_model.Lasso(alpha=alpha)).fit(x_train_df, y_train_df)

In [ ]:
selected_feats_names = list(set(selector.get_feature_names_out()))
selected_feats_names

In [ ]:
x_train_feats = x_train_df[selected_feats_names].values
x_query_feats = x_query_df[selected_feats_names].values

# train a classifier: logistic regression classifier
lr_classifier = LogisticRegression(max_iter=1000).fit(X=x_train_feats, y=y_train_df)

In [ ]:
## predict test data

# predict labels using the trained classifier
pred_labels = lr_classifier.predict(x_query_feats)

# accuracy of the test data
print(f"accuracy of the query data: {np.mean(pred_labels == y_query_df)}")